# PARAMETER ESTIMATION of $\beta_{int}, \beta_{ext}, \beta_{pen}$ FOR SWINE FARROW-TO-FINISH MODEL

## Detailed explanation of each cell code can be read on the markdown on top of them.

### Initialization of packages for ABC SMC parameter estimation and EMULSION. Install if necessary.

!pip install pyabc emulsion==1.2rc5

### Libraries and databases

This cell sets up the environment and configurations for running a simulation using the `pyabc` library, which is used for Approximate Bayesian Computation (ABC). Below is a step-by-step explanation of what each part of the code does:

1. **Importing Libraries**:
    - `os`, `shutil`: For file and directory operations.
    - `pandas as pd`: For data manipulation and analysis.
    - `matplotlib.pyplot as plt`, `numpy as np`: For plotting and numerical operations.
    - `pyabc`, `emulsion`: For ABC simulations and related tasks.
    - `datetime`: For handling date and time.

2. **Setting Plot Parameters**:
    ```python
    pyabc.settings.set_figure_params('pyabc')
    ```
    This line configures the plotting parameters for `pyabc` to ensure that the plots are beautified and consistent.

3. **Jupyter Notebook Specific Command**:
    ```python
    %matplotlib inline
    ```
    This command is specific to Jupyter notebooks and ensures that plots are displayed inline within the notebook. If running the script outside a Jupyter notebook, this line can be removed.

4. **Importing Specific Classes and Functions from `pyabc`**:
    - `ABCSMC`, `RV`, `Distribution`, `LocalTransition`, `MedianEpsilon`: Core components for setting up and running ABC simulations.
    - `plot_data_callback`, `plot_kde_2d`: Functions for visualizing data and results.
    - `AdaptivePNormDistance`: A distance metric used in ABC.
    - `MulticoreEvalParallelSampler`, `MulticoreParticleParallelSampler`, `SingleCoreSampler`, `RedisEvalParallelSampler`: Different samplers for parallel execution of simulations.

5. **Configuring the Redis Sampler**:
    ```python
    redis_sampler = RedisEvalParallelSampler(host="111.111.111.111", port=6379)
    ```
    This line sets up a Redis-based parallel sampler, which allows for distributed computation across multiple cores or machines.

6. **Getting the Current Date and Time**:
    ```python
    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    ```
    This line retrieves the current date and time in the format `YYYY-MM-DD-HH-MM-SS`.

7. **Defining a Persistent Database Path**:
    ```python
    persistent_db_dir = "persistent_db"
    os.makedirs(persistent_db_dir, exist_ok=True)
    ```
    This creates a directory named `persistent_db` if it does not already exist. This directory will be used to store the database files.

8. **Creating the Database Path with the Current Date**:
    ```python
    db_path = "sqlite:///" + os.path.join(persistent_db_dir, f"test_{current_date}_euclidean.db")
    ```
    This constructs the path for the SQLite database file, including the current date and time in the filename to ensure uniqueness.

9. **Using the Database Path in `pyabc` Settings**:
    The `db_path` variable can now be used in the `pyabc` settings to specify where the simulation results should be stored.

This setup ensures that your ABC simulations are configured correctly, with results stored in a uniquely named database file for each run.

In [1]:
import os
import pandas as pd
import shutil

import matplotlib.pyplot as plt
import numpy as np

import pyabc
import emulsion

from datetime import datetime

pyabc.settings.set_figure_params('pyabc')  # for beautified plots

# Note: %matplotlib inline is specific to Jupyter notebooks and should be used there
# If you are running this script outside a Jupyter notebook, you can remove the line
# %matplotlib inline

from pyabc import ABCSMC, RV, Distribution, LocalTransition, MedianEpsilon
from pyabc.visualization import plot_data_callback, plot_kde_2d
from pyabc.distance import AdaptivePNormDistance
from pyabc.sampler import MulticoreEvalParallelSampler, MulticoreParticleParallelSampler, SingleCoreSampler
from pyabc.sampler import RedisEvalParallelSampler

# Configure Redis sampler
redis_sampler = RedisEvalParallelSampler(host="111.111.111.111", port=6379)

# Get the current date in YYYY-MM-DD format
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Define a persistent database path with the date included
persistent_db_dir = "persistent_db"
os.makedirs(persistent_db_dir, exist_ok=True)

# Create the database path with the current date in the filename
db_path = "sqlite:///" + os.path.join(persistent_db_dir, f"test_{current_date}_euclidean.db")

# Now you can use `db_path` in your pyabc settings

### Function that renames a file with a counter for the outputs of the simulation

This function renames a specified file by appending a counter to its name, ensuring the new name is unique within its directory. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    - `os`: For interacting with the operating system, such as file and directory operations.
    - `time`, `uuid`: Imported but not used in this function.

2. **Function Definition**:
    ```python
    def rename_file_with_counter(input_file):
    ```
    This defines the function `rename_file_with_counter` which takes one argument, `input_file`, representing the path to the file to be renamed.

3. **Checking File Existence**:
    ```python
    if os.path.exists(input_file):
    ```
    This checks if the specified file exists. If it does not, the function prints an error message and returns `None`.

4. **Splitting Directory and Filename**:
    ```python
    directory, filename = os.path.split(input_file)
    file_extension = filename.split('.')[-1]
    ```
    This splits the input file path into the directory and the filename. It also extracts the file extension.

5. **Listing and Sorting Files in the Directory**:
    ```python
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    files.sort()
    ```
    This creates a sorted list of all files in the same directory as the input file.

6. **Finding the Index of the Input File**:
    ```python
    index = files.index(filename)
    ```
    This finds the index of the input file in the sorted list of files.

7. **Generating the New Filename**:
    ```python
    counter = index + 1
    new_name = f"{counter:03d}.{file_extension}"
    new_path = os.path.join(directory, new_name)
    ```
    This generates a new filename using a counter based on the file's index in the sorted list. The counter is formatted to be three digits with leading zeros.

8. **Renaming the File**:
    ```python
    os.rename(input_file, new_path)
    print(f"File '{filename}' renamed to '{new_name}'.")
    ```
    This renames the file to the new name and prints a confirmation message.

9. **Returning the New Path**:
    ```python
    return new_path
    ```
    This returns the new path of the renamed file.

10. **Handling Non-Existent Files**:
    ```python
    else:
        print("File does not exist or is not valid.")
        return None
    ```
    If the file does not exist, the function prints an error message and returns `None`.

This function ensures that the renamed file has a unique name within its directory by appending a counter based on its position in a sorted list of files.


In [2]:
import os
import time
import uuid

def rename_file_with_counter(input_file):
    if os.path.exists(input_file):
        directory, filename = os.path.split(input_file)
        file_extension = filename.split('.')[-1]
        
        # Get a list of files in the same directory
        files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        files.sort()
        
        # Find the index of the input file in the sorted list
        index = files.index(filename)
        
        # Counter based on the index
        counter = index + 1
        
        new_name = f"{counter:03d}.{file_extension}"  # Using 3 digits with leading zeros
        new_path = os.path.join(directory, new_name)
        
        os.rename(input_file, new_path)
        print(f"File '{filename}' renamed to '{new_name}'.")
        
        return new_path
    else:
        print("File does not exist or is not valid.")
        return None

### Functions that delete a file, a dolder, and create output directory whic is unique using time stamp and UUID. The deletion saves space.

This code snippet includes three functions: `delete_file`, `delete_folder`, and `create_output_directory`. Each function performs a specific file or directory operation. Below is a detailed explanation of each function:

1. **Function: `delete_file`**
    ```python
    def delete_file(file_path):
        try:
            os.remove(file_path)
            print(f"File '{file_path}' has been deleted.")
        except FileNotFoundError:
            print(f"File '{file_path}' not found. Deletion failed.")
    ```
    - **Purpose**: Deletes a specified file.
    - **Parameters**: `file_path` - The path to the file to be deleted.
    - **Process**:
        - Attempts to delete the file using `os.remove(file_path)`.
        - If the file is successfully deleted, it prints a confirmation message.
        - If the file is not found, it catches the `FileNotFoundError` and prints an error message.

2. **Function: `delete_folder`**
    ```python
    def delete_folder(folder_path):
        try:
            # Use shutil.rmtree() to recursively remove the folder and its contents
            shutil.rmtree(folder_path)
            print(f"Folder '{folder_path}' has been deleted.")
        except Exception as e:
            print(f"Error: {e}")
    ```
    - **Purpose**: Deletes a specified folder and all its contents.
    - **Parameters**: `folder_path` - The path to the folder to be deleted.
    - **Process**:
        - Attempts to delete the folder and its contents using `shutil.rmtree(folder_path)`.
        - If the folder is successfully deleted, it prints a confirmation message.
        - If an error occurs, it catches the exception and prints an error message.

3. **Function: `create_output_directory`**
    ```python
    def create_output_directory():
        # Generate a unique directory name using a timestamp and a UUID
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        unique_id = str(uuid.uuid4())[:8]  # Extract the first 8 characters of the UUID
        directory_name = f"output_{timestamp}_{unique_id}"

        # Create the output directory
        os.mkdir(directory_name)

        return directory_name
    ```
    - **Purpose**: Creates a uniquely named output directory.
    - **Parameters**: None.
    - **Process**:
        - Generates a unique directory name using the current timestamp and a UUID.
        - The timestamp is formatted as `YYYY-MM-DD-HH-MM-SS`.
        - The UUID is truncated to the first 8 characters.
        - Combines the timestamp and UUID to form the directory name.
        - Creates the directory using `os.mkdir(directory_name)`.
        - Returns the name of the created directory.

### Example Usage

To delete a folder:
```python
folder_to_delete = "path/to/your/folder"
delete_folder(folder_to_delete)


In [3]:
def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f"File '{file_path}' has been deleted.")
    except FileNotFoundError:
        print(f"File '{file_path}' not found. Deletion failed.")

def delete_folder(folder_path):
    try:
        # Use shutil.rmtree() to recursively remove the folder and its contents
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' has been deleted.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
#folder_to_delete = "path/to/your/folder"
#delete_folder(folder_to_delete)

def create_output_directory():
    # Generate a unique directory name using a timestamp and a UUID
    timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    unique_id = str(uuid.uuid4())[:8]  # Extract the first 8 characters of the UUID
    directory_name = f"output_{timestamp}_{unique_id}"

    # Create the output directory
    os.mkdir(directory_name)

    return directory_name

# Example usage:
#output_dir = create_output_directory()
#print(f"Created output directory: {output_dir}")

### EMULSION model of a farrow-to-finish swine farm

This function sets up and runs a simulation using the `emulsion` library, which is designed for modeling and simulating epidemiological processes. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    - `docopt`: For parsing command-line arguments.
    - `Path` from `pathlib`: For handling file paths.
    - `pandas as pd`: For data manipulation and analysis.
    - `emulsion.__main__ as em`: For accessing the main functionality of the `emulsion` library.

2. **Function Definition**:
    ```python
    def emul(trans=5, trans_betwn_herds=1, trans_neighbor=1, delta=1, time=33, reps=1):
    ```
    This defines the function `emul` with default parameters for various transmission rates, time steps, and repetitions.

3. **Parsing Command-Line Arguments**:
    ```python
    doc = em.__doc__
    args = docopt(doc, help=False, argv=['-V'])
    ```
    This retrieves the docstring from the `emulsion` module, which contains the syntax for the command, and parses it using `docopt`.

4. **Setting Model and Run Options**:
    ```python
    args['MODEL'] = 'swine_model_weekly-12fatpens-FINAL.yaml'
    args['run'] = True
    args['--silent'] = True
    ```
    These lines set the model to be used (`SWINE_MODEL.yaml`), indicate that simulations should be run, and set the silent mode to suppress output.

5. **Setting Simulation Parameters**:
    ```python
    args['--param'] = ['transmission_rate=' + str(trans), 
                       'between_herd_trans=' + str(trans_betwn_herds), 
                       'neighboring_herd_trans=' + str(trans_neighbor)]
    args['--runs'] = str(reps)
    args['--time'] = str(time)
    ```
    These lines set the specific parameters for the simulation, including transmission rates, the number of stochastic repetitions, and the number of time steps.

6. **Creating a Unique Output Directory**:
    ```python
    output_dir = create_output_directory()
    args['--output-dir'] = str(output_dir)
    ```
    This creates a unique output directory using the `create_output_directory` function and sets it in the arguments.

7. **Running the Simulation**:
    ```python
    em.main(args)
    ```
    This runs the simulation with the specified parameters.

8. **Reading and Processing Output Data**:
    ```python
    output = pd.read_csv(Path(args['--output-dir']).joinpath('counts.csv'))
    data1 = output[output['population_id'] == 4]
    data1 = data1.reset_index()
    data1.fillna(0, inplace=True)
    data = pd.DataFrame(data1['nb_of_sampled_I_Jf'].values)
    ```
    This reads the output data from the simulation, filters it for a specific population, resets the index, fills any missing values with zero, and extracts the relevant data into a new DataFrame.

9. **Deleting the Output Directory**:
    ```python
    delete_folder(output_dir)
    ```
    This deletes the output directory and its contents to clean up after the simulation.

10. **Renaming the DataFrame Column**:
    ```python
    new_column_name = 'I'
    data = data.rename(columns={0: new_column_name})
    ```
    This renames the single column in the DataFrame to 'I'.

11. **Returning the Data**:
    ```python
    return data
    ```
    This returns the processed data as a DataFrame.

### Example Usage

To run the `emul` function with default parameters:
```python
result = emul()
print(result)


In [4]:
from docopt import docopt # to parse the command-line arguments
from pathlib import Path
import pandas as pd
import emulsion.__main__ as em

def emul(trans = 5,
         trans_betwn_herds = 1,
         trans_neighbor = 1,
         delta = 1,
         time = 33,
         reps = 1,
         #nb_contacts = 0
        ):
    
    doc = em.__doc__ # the docstring contains the syntax of the emulsion command
    # parse arguments and get the dictionary of available sub-commands and options
    args = docopt(doc, help=False, argv=['-V'])
    # define the name of the model
    #args['MODEL'] = 'swine_model_weekly-12fatpens-weight-FINAL.yaml'
    args['MODEL'] = 'SWINE_MODEL.yaml'
    # define that you want to run the simulations
    args['run'] = True
    # define other options...
    args['--silent'] = True
    # ...including specific parameter values...
    #args['--param'] = ['ext_inf_trans=' + str(ext_inf_trans)]
    args['--param'] = ['transmission_rate=' + str(trans), 
                       'between_herd_trans=' + str(trans_betwn_herds), 
                       'neighboring_herd_trans=' + str(trans_neighbor)
                      ]
    #                   , 'init_proba_mat_imm=' + str(init_proba_mat_imm),
    #                   'init_portion_I=' + str(init_portion_I)]
    # ... or the number of stochastic repetitions...
    args['--runs'] = str(reps)
    # ... or the number of time steps...
    args['--time'] = str(time)
    # ...run the model in deterministic mode
    # args['--deterministic']
    # ...or the directory where the output file (counts.csv) is located (hence, 1 output dir per parameter set is a good practice if you intend to keep them all or are running simulations in parallel)
    #create unique output directory
    output_dir = create_output_directory()
    args['--output-dir'] = str(output_dir)
    # then run the simulation for the specified parameters
    em.main(args)
    # when the simulation is over, retrieve results in a Pandas data frame
    #path of the output file
    #path = "{}/counts.csv"
    #read output file
    #output_file = path.format(output_dir)
    #output = pd.read_csv(output_file)
    output = pd.read_csv(Path(args['--output-dir']).joinpath('counts.csv'))
    # have a look at the content
    #results_dataframe.head()
    
    #extract and transform data from ouput
    data1 = output[output['population_id']==4]
    data1 = data1.reset_index()
    #data1 = data1[data1.index%7 == 0]
    data1.fillna(0, inplace=True)
    
    # filter data to include only indices from 163 to 180
    #data1 = data1[(data1.index >= 17) & (data1.index <= 34)]
    #data1.fillna(0, inplace=True)
    
    #save the extracted data in a dataframe
    data = pd.DataFrame(data1['nb_of_sampled_I_Jf'].values)
    #data = pd.DataFrame(data1['I'].values)
    
    #delete output if necessary
    delete_folder(output_dir)
    
    # Rename the single column
    new_column_name = 'I'
    data = data.rename(columns={0: new_column_name})
    
    return data

### A Wrapper for the output of the EMULSION model that can be called by pyABC.

This function serves as a wrapper around the `emul` function, allowing you to pass parameters to it and return the results in a structured format. Below is a detailed breakdown of the code:

1. **Function Definition**:
    ```python
    def model(parameter):
    ```
    This defines the function `model` which takes one argument, `parameter`, expected to be a dictionary of parameters.

2. **Calling the `emul` Function**:
    ```python
    res = emul(**parameter)
    ```
    This line calls the `emul` function, unpacking the `parameter` dictionary into keyword arguments using `**parameter`. The result of the `emul` function is stored in the variable `res`.

3. **Returning the Result**:
    ```python
    return {"data": res}
    ```
    This returns a dictionary with a single key `"data"` containing the result from the `emul` function.

### Example Usage

To use the `model` function, you would pass a dictionary of parameters:
```python
parameters = {
    "trans": 5,
    "trans_betwn_herds": 1,
    "trans_neighbor": 1,
    "delta": 1,
    "time": 33,
    "reps": 1
}

result = model(parameters)
print(result)


In [5]:
def model(parameter):
    res = emul(**parameter)
    return {"data": res}

### Trajectory from initial guess of the parameter values

This cell captures the output of the `model` function call and suppresses its display in the notebook. Below is a detailed breakdown of the code:

1. **Magic Command: `%%capture --no-display`**:
    ```python
    %%capture --no-display
    ```
    - **Purpose**: This Jupyter notebook magic command captures the output of the cell, including stdout and stderr, and prevents it from being displayed in the notebook.
    - **Option**: `--no-display` ensures that the captured output is not displayed.

2. **Calling the `model` Function**:
    ```python
    init_trajectory = model({'trans': 5.0, 'trans_betwn_herds': 1.0, 'trans_neighbor': 1.0})
    ```
    - **Function Call**: This line calls the `model` function with a dictionary of parameters.
    - **Parameters**:
        - `trans`: Transmission rate set to 5.0.
        - `trans_betwn_herds`: Transmission rate between herds set to 1.0.
        - `trans_neighbor`: Transmission rate between neighboring herds set to 1.0.
    - **Result**: The result of the `model` function is stored in the variable `init_trajectory`.

In [ ]:
%%capture --no-display 
init_trajectory = model({'trans': 5.0, 'trans_betwn_herds': 1.0, 'trans_neighbor': 1.0})

### Explanation of the Code Snippet

This code snippet processes the output of the `model` function and extracts specific data from it. Below is a detailed breakdown of the code:

1. **Extracting Data from the Dictionary**:
    ```python
    init_trajectory = init_trajectory['data']
    ```
    This line extracts the value associated with the key `'data'` from the `init_trajectory` dictionary. The `init_trajectory` variable now holds the DataFrame returned by the `model` function.

2. **Extracting the 'I' Column Values**:
    ```python
    init_trajectory = init_trajectory['I'].values
    ```
    This line extracts the values from the `'I'` column of the DataFrame and converts them into a NumPy array. The `init_trajectory` variable now holds this array.

3. **Checking the Type of `init_trajectory`**:
    ```python
    type(init_trajectory)
    ```
    This line checks the type of the `init_trajectory` variable. Since the previous line converts the DataFrame column to a NumPy array, this should return `<class 'numpy.ndarray'>`.

4. **Displaying the `init_trajectory` Array**:
    ```python
    init_trajectory
    ```
    This line displays the contents of the `init_trajectory` array. In a Jupyter notebook, this will output the array values.

In [ ]:
init_trajectory = init_trajectory['data']
init_trajectory = init_trajectory['I'].values
type(init_trajectory)
init_trajectory

### Reads observation data from a CSV file if available.

In [ ]:
measurement_data = pd.read_csv("infected_data.csv")
measurement_data = measurement_data['nb_I'].values
#measurement_data = measurement_data['data']
#measurement_data = measurement_data['I'].values
measurement_data

### Produces synthetic observation data

This code snippet runs a model simulation 100 times, processes the results, and visualizes the trajectories along with the median and the closest output to the median. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    ```python
    import numpy as np
    from scipy.spatial.distance import euclidean
    import matplotlib.pyplot as plt
    ```
    - `numpy`: For numerical operations and handling arrays.
    - `euclidean` from `scipy.spatial.distance`: For calculating Euclidean distance.
    - `matplotlib.pyplot`: For plotting the results.

2. **Initializing a List to Store Results**:
    ```python
    results = []
    ```
    This initializes an empty list to store the results of each model run.

3. **Running the Model 100 Times**:
    ```python
    for _ in range(100):
        measurement_data = model({'trans': 5.0, 'trans_betwn_herds': 1.0, 'trans_neighbor': 1.0})
        measurement_data = measurement_data['data']
        measurement_data = measurement_data['I'].values
        results.append(measurement_data)
    ```
    - This loop runs the `model` function 100 times with specified parameters.
    - The results of each run are extracted, specifically the `'I'` column values, and appended to the `results` list.

4. **Converting Results to a NumPy Array**:
    ```python
    results = np.array(results)
    ```
    This converts the list of results into a NumPy array for easier manipulation and analysis.

5. **Computing the Median of the Results**:
    ```python
    median_measurement_data = np.median(results, axis=0)
    ```
    This calculates the median of the results along the specified axis (time steps).

6. **Finding the Output Closest to the Median**:
    ```python
    closest_output = min(results, key=lambda x: euclidean(x, median_measurement_data))
    ```
    This finds the output that is closest to the median using the Euclidean distance metric.

7. **Plotting the Results**:
    ```python
    plt.figure(figsize=(10, 6))

    for result in results:
        plt.plot(result, color='lightgray', alpha=0.5)

    plt.plot(median_measurement_data, color='blue', label='Median', linewidth=2)
    plt.plot(closest_output, color='red', label='Closest Output', linewidth=2)

    plt.xlabel('Time')
    plt.ylabel('Infected Pens')
    plt.title('Trajectories of Outputs with Median and Closest Output')
    plt.legend()
    plt.savefig('100model_runs_&_median_betas_511.png', dpi=300, bbox_inches='tight')
    plt.show()
    ```
    - **Figure Setup**: Initializes a new figure with specified size.
    - **Plotting Trajectories**: Plots each trajectory in light gray with some transparency.
    - **Plotting Median**: Plots the median trajectory in blue.
    - **Plotting Closest Output**: Plots the closest output to the median in red.
    - **Labels and Title**: Adds labels for the x and y axes and a title.
    - **Legend**: Adds a legend to distinguish between the median and the closest output.
    - **Saving the Figure**: Saves the plot as a PNG file with specified resolution and bounding box.
    - **Displaying the Plot**: Displays the plot.



In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Initialize a list to store the results
results = []

# Run the model 100 times
for _ in range(100):
    measurement_data = model({'trans': 5.0, 'trans_betwn_herds': 1.0, 'trans_neighbor': 1.0})
    measurement_data = measurement_data['data']
    measurement_data = measurement_data['I'].values
    results.append(measurement_data)

# Convert results to a numpy array
results = np.array(results)

# Compute the median of the results
median_measurement_data = np.median(results, axis=0)

# Find the output closest to the median using Euclidean distance
closest_output = min(results, key=lambda x: euclidean(x, median_measurement_data))

# Plot all the trajectories, the median, and the closest output
plt.figure(figsize=(10, 6))

# Plot each trajectory
for result in results:
    plt.plot(result, color='lightgray', alpha=0.5)

# Plot the median
plt.plot(median_measurement_data, color='blue', label='Median', linewidth=2)

# Plot the closest output
plt.plot(closest_output, color='red', label='Closest Output', linewidth=2)

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Infected Pens')
plt.title('Trajectories of Outputs with Median and Closest Output')

# Add a legend
plt.legend()

# Save the figure
plt.savefig('100model_runs_&_median_betas_511.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

### Time axis values for plotting

In [12]:
measurement_times = np.arange(len(measurement_data))

### Plot comparison between model with initial guess of parameters and observation data

In [ ]:
plt.plot(init_trajectory, color="C0", label='Simulation')
plt.plot(measurement_times, measurement_data, color="C1", label='Data')
plt.xlabel('Week $t$')
plt.ylabel('No. of infected fattening pens')
plt.title('Initial parameters fit')
plt.legend()
plt.show()

# PARAMETER ESTIMATION

### Distance used by ABC SMC to test closeness of observed and simulated data per parameter value using a least square measure.

This code snippet sets up various distance metrics for use in Approximate Bayesian Computation (ABC) using the `pyabc` library. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    ```python
    import logging
    import tempfile
    ```
    - `logging`: For logging debug information.
    - `tempfile`: For creating temporary files.

2. **Defining a Custom Euclidean Distance Function**:
    ```python
    def distance_sqrd_error(x, y):
        diff = x["data"]["I"] - y["data"]["I"]
        dist = np.sqrt(np.sum(diff**2))
        return dist
    ```
    - **Purpose**: Computes the Euclidean distance between two sets of data.
    - **Parameters**: `x` and `y` are dictionaries containing the data.
    - **Process**: Calculates the difference between the `I` values in the data, squares the differences, sums them, and takes the square root to get the Euclidean distance.

3. **Setting Up Logging for Debugging**:
    ```python
    df_logger = logging.getLogger("ABC.Distance")
    df_logger.setLevel(logging.DEBUG)
    ```
    - **Purpose**: Configures a logger for debugging distance calculations.
    - **Logger Name**: `"ABC.Distance"`.
    - **Log Level**: `DEBUG`, which captures detailed information for debugging.

4. **Defining Built-in Distance Metrics**:
    ```python
    distance_euclid = pyabc.PNormDistance(p=2)
    distance_man = pyabc.PNormDistance(p=1)
    ```
    - **Euclidean Distance**: `distance_euclid` uses the L2-norm (`p=2`).
    - **Manhattan Distance**: `distance_man` uses the L1-norm (`p=1`).

5. **Setting Up Adaptive Distance Metrics**:
    ```python
    scale_log_file = tempfile.mkstemp(suffix=".json")

    distance_adaptive = pyabc.AdaptivePNormDistance(
        p=2,
        scale_function=pyabc.distance.pcmad,
        scale_log_file=scale_log_file,
    )
    ```
    - **Purpose**: Configures an adaptive distance metric using the L2-norm (`p=2`).
    - **Scaling Function**: `pyabc.distance.pcmad`, which scales the distance based on the population coefficient of median absolute deviation (PCMAD).
    - **Log File**: Stores scaling information in a temporary JSON file.

6. **Setting Up L1-norm Adaptive Distance Metric**:
    ```python
    scale_log_file_l1 = tempfile.mkstemp(suffix=".json")

    distance_adaptive_l1 = pyabc.AdaptivePNormDistance(
        p=1,
        scale_function=pyabc.distance.pcmad,
        scale_log_file=scale_log_file_l1,
    )
    ```
    - **Purpose**: Configures an adaptive distance metric using the L1-norm (`p=1`).
    - **Scaling Function**: `pyabc.distance.pcmad`.
    - **Log File**: Stores scaling information in a separate temporary JSON file.

This setup provides multiple distance metrics for use in ABC, including custom, built-in, and adaptive metrics, allowing for flexible and robust distance calculations in your simulations.


In [ ]:
import logging
import tempfile

# Euclidean distance
def distance_sqrd_error(x, y):
    diff = x["data"]["I"] - y["data"]["I"]
    dist = np.sqrt(np.sum(diff**2))
    return dist

# for debugging
df_logger = logging.getLogger("ABC.Distance")
df_logger.setLevel(logging.DEBUG)

#Euclidean distance (built-in pyABC)
distance_euclid = pyabc.PNormDistance(p=2)

#manhattan distance 
distance_man = pyabc.PNormDistance(p=1)

scale_log_file = tempfile.mkstemp(suffix=".json")[1]

distance_adaptive = pyabc.AdaptivePNormDistance(
    p=2,
    scale_function=pyabc.distance.pcmad,  # method by which to scale
    scale_log_file=scale_log_file,
)

# Using L1-norm distance with PCMAD

scale_log_file_l1 = tempfile.mkstemp(suffix=".json")[1]

distance_adaptive_l1 = pyabc.AdaptivePNormDistance(
    p=1,  # new, previously p=2
    scale_function=pyabc.distance.pcmad,
    scale_log_file=scale_log_file_l1,
)

### Prior distribution or interval where the values of a parameter lie.

This code snippet sets up a prior distribution for the parameters of a model using the `pyabc` library. Below is a detailed breakdown of the code:

1. **Defining Maximum Values for Parameters**:
    ```python
    max_trans = 4
    max_trans_betwn_herds = 4
    max_trans_neighbor = 4
    ```
    These lines define the maximum values for the parameters `trans`, `trans_betwn_herds`, and `trans_neighbor`.

2. **Creating a Prior Distribution**:
    ```python
    parameter_prior = Distribution(
        trans=RV("uniform", 0.0, max_trans),
        trans_betwn_herds=RV("uniform", 0.0, max_trans_betwn_herds),
        trans_neighbor=RV("uniform", 0.0, max_trans_neighbor),
    )
    ```
    - **Purpose**: Sets up a prior distribution for the parameters using uniform distributions.
    - **Parameters**:
        - `trans`: Uniform distribution between 0.0 and `max_trans`.
        - `trans_betwn_herds`: Uniform distribution between 0.0 and `max_trans_betwn_herds`.
        - `trans_neighbor`: Uniform distribution between 0.0 and `max_trans_neighbor`.

3. **Getting Parameter Names**:
    ```python
    parameter_prior.get_parameter_names()
    ```
    This line retrieves the names of the parameters in the prior distribution. It returns a list of parameter names: `['trans', 'trans_betwn_herds', 'trans_neighbor']`.

### Example Usage

To set up and retrieve the parameter names:
```python
max_trans = 4
max_trans_betwn_herds = 4
max_trans_neighbor = 4
parameter_prior = Distribution(
    trans=RV("uniform", 0.0, max_trans),
    trans_betwn_herds=RV("uniform", 0.0, max_trans_betwn_herds),
    trans_neighbor=RV("uniform", 0.0, max_trans_neighbor),
)
parameter_names = parameter_prior.get_parameter_names()
print(parameter_names)  # Output: ['trans', 'trans_betwn_herds', 'trans_neighbor']


In [ ]:
max_trans = 4
max_trans_betwn_herds = 4
max_trans_neighbor = 4
parameter_prior = Distribution(
    trans=RV("uniform", 0.0, max_trans),
    trans_betwn_herds=RV("uniform", 0.0, max_trans_betwn_herds),
    trans_neighbor=RV("uniform", 0.0, max_trans_neighbor),
)
parameter_prior.get_parameter_names()

### Synthetic observation data

In [16]:
# Convert to DataFrame with column name 'I', then to a dict of the form {"data": }
measurement_data = pd.DataFrame(measurement_data, columns=['I'])
measurement_data = {"data": measurement_data}
init_trajectory = pd.DataFrame(init_trajectory, columns=['I'])
init_trajectory = {"data": init_trajectory}

### ABC SMC Simulation function

This code snippet sets up an Approximate Bayesian Computation Sequential Monte Carlo (ABC-SMC) algorithm using the `pyabc` library. Below is a detailed breakdown of the code:

1. **Importing the Adaptive Population Size Strategy**:
    ```python
    from pyabc.populationstrategy import AdaptivePopulationSize
    ```
    This imports the `AdaptivePopulationSize` strategy from the `pyabc` library, which allows the population size to adapt based on the progress of the ABC-SMC algorithm.

2. **Setting Up the ABC-SMC Algorithm**:
    ```python
    abc = ABCSMC(
        model, 
        parameter_prior, 
        distance_adaptive_l1, 
        50,
        #sampler=MulticoreParticleParallelSampler(n_procs=4),
    )
    ```
    - **Purpose**: Initializes the ABC-SMC algorithm with the specified model, prior distribution, distance metric, and population size.
    - **Parameters**:
        - `model`: The model function to be used in the ABC-SMC algorithm.
        - `parameter_prior`: The prior distribution of the parameters.
        - `distance_adaptive_l1`: The distance metric to be used, in this case, an adaptive L1-norm distance.
        - `50`: The initial population size for the ABC-SMC algorithm.
        - `sampler`: (Commented out) An optional parameter to specify the sampler. Here, `MulticoreParticleParallelSampler` is commented out, which would allow parallel processing using multiple cores.


In [ ]:
from pyabc.populationstrategy import AdaptivePopulationSize

abc = ABCSMC(
    model, 
    parameter_prior, 
    distance_adaptive_l1, 
    50,
    #sampler=MulticoreParticleParallelSampler(n_procs=4),
)

### Define pyabc ID

This code snippet initializes a new ABC-SMC run and stores the initial measurement data in the database. Below is a detailed breakdown of the code:

1. **Initializing a New ABC-SMC Run**:
    ```python
    abc_id = abc.new(db_path, measurement_data)
    ```
    - **Purpose**: This line initializes a new run of the ABC-SMC algorithm.
    - **Parameters**:
        - `db_path`: The path to the database where the results will be stored. This should be a valid database URI, such as an SQLite database path.
        - `measurement_data`: The observed data against which the model simulations will be compared. This data is used to compute the distance between the observed data and the simulated data.
    - **Return Value**: The `abc.new` method returns an identifier (`abc_id`) for the new ABC-SMC run. This identifier can be used to reference the run in subsequent operations.


In [ ]:
abc_id = abc.new(db_path, measurement_data)

### Function to run the ABC SMC algorithm.

This cell runs the ABC-SMC algorithm while capturing and suppressing its output in the notebook. Below is a detailed breakdown of the code:

1. **Magic Command: `%%capture --no-display`**:
    ```python
    %%capture --no-display
    ```
    - **Purpose**: This Jupyter notebook magic command captures the output of the cell, including stdout and stderr, and prevents it from being displayed in the notebook.
    - **Option**: `--no-display` ensures that the captured output is not shown.

2. **Running the ABC-SMC Algorithm**:
    ```python
    h = abc.run(minimum_epsilon=0.001, max_nr_populations=5)
    ```
    - **Purpose**: This line runs the ABC-SMC algorithm with specified stopping criteria.
    - **Parameters**:
        - `minimum_epsilon=0.001`: The algorithm stops when the distance metric (epsilon) reaches this minimum value.
        - `max_nr_populations=5`: The algorithm stops after generating this maximum number of populations.
    - **Return Value**: The result of the run is stored in the variable `h`, which contains the history of the ABC-SMC run, including the accepted particles and their weights.


In [ ]:
%%capture --no-display
h = abc.run(minimum_epsilon=0.001, max_nr_populations=5)

## Visualization

### Progression of the parameter estimation for each parameter with a 95% credible interval

This code snippet uses the `pyabc` library to plot credible intervals for the parameters of an ABC-SMC model. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    ```python
    import pyabc
    import matplotlib.pyplot as plt
    ```
    - `pyabc`: For performing and visualizing Approximate Bayesian Computation (ABC).
    - `matplotlib.pyplot`: For creating plots.

2. **Defining Plot Parameters**:
    ```python
    plot_params = {
        "history": h,
        "m": 0,  # Model ID
        #"ts": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Time points to plot
        "par_names": [
            'trans', 
            'trans_betwn_herds', 
            'trans_neighbor', 
            #'ext_inf'
        ],  # Parameter names to plot
        "levels": [0.95],  # Confidence intervals
        "colors": ["blue"],  # Colors for the error bars
        "show_mean": True,  # Whether to show mean
        "color_mean": "green",  # Color for the mean
        "show_kde_max": True,  # Whether to show KDE max
        "color_kde_max": "red",  # Color for KDE max
        "show_kde_max_1d": False,  # Whether to show KDE max 1D
        "color_kde_max_1d": "purple",  # Color for KDE max 1D
        "size": (10, 8),  # Size of the plot
    }
    ```
    - **`history`**: The `pyabc` history object containing the results of the ABC-SMC run.
    - **`m`**: The model ID to plot (0 in this case).
    - **`par_names`**: List of parameter names to include in the plot.
    - **`levels`**: Confidence intervals to plot (95% in this case).
    - **`colors`**: Colors for the error bars.
    - **`show_mean`**: Whether to show the mean of the parameters.
    - **`color_mean`**: Color for the mean line.
    - **`show_kde_max`**: Whether to show the KDE (Kernel Density Estimate) maximum.
    - **`color_kde_max`**: Color for the KDE maximum line.
    - **`show_kde_max_1d`**: Whether to show the KDE maximum in 1D.
    - **`color_kde_max_1d`**: Color for the KDE maximum in 1D.
    - **`size`**: Size of the plot.

3. **Plotting Credible Intervals**:
    ```python
    pyabc.visualization.plot_credible_intervals(**plot_params)
    ```
    This line uses the `plot_credible_intervals` function from `pyabc.visualization` to create the plot based on the specified parameters.

4. **Displaying the Plot**:
    ```python
    plt.show()
    ```
    This line displays the plot in the Jupyter notebook.

In [ ]:
import pyabc
import matplotlib.pyplot as pltg

# pyABC History object named 'h'
# Update the parameters as needed
plot_params = {
    "history": h,
    "m": 0,  # Model ID
    #"ts": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Time points to plot
    "par_names": [
        'trans', 
        'trans_betwn_herds', 
        'trans_neighbor', 
        #'ext_inf'
    ],  # Parameter names to plot
    "levels": [0.95],  # Confidence intervals
    "colors": ["blue"],  # Colors for the error bars
    "show_mean": True,  # Whether to show mean
    "color_mean": "green",  # Color for the mean
    "show_kde_max": True,  # Whether to show KDE max
    "color_kde_max": "red",  # Color for KDE max
    "show_kde_max_1d": False,  # Whether to show KDE max 1D
    "color_kde_max_1d": "purple",  # Color for KDE max 1D
    "size": (10, 8),  # Size of the plot
}

# Plot credible intervals
pyabc.visualization.plot_credible_intervals(**plot_params)

# Show the plot
plt.show()

### Posterior distribution of the parameters for all generations

This code snippet uses the `pyabc` library to plot a KDE (Kernel Density Estimate) matrix for the parameters of an ABC-SMC model, customizing the plot for better readability in publications. Below is a detailed breakdown of the code:

1. **Importing Libraries**:
    ```python
    from pyabc.visualization import plot_kde_matrix
    import numpy as np
    import matplotlib.pyplot as plt
    ```
    - `plot_kde_matrix`: For plotting KDE matrices.
    - `numpy`: For numerical operations.
    - `matplotlib.pyplot`: For creating plots.

2. **Defining Estimated Parameter Values**:
    ```python
    trans_est = 1
    trans_betwn_herds_est = 1
    trans_neighbor_est = 1
    ```
    These lines define the estimated values for the parameters `trans`, `trans_betwn_herds`, and `trans_neighbor`.

3. **Iterating Over Populations**:
    ```python
    for i in range(h.max_t, h.max_t+1):
        # Estimated parameters
        df, w = h.get_distribution(m=0)
        true_trans = np.average(df['trans'].values, weights=w)
        true_trans_betwn_herds = np.average(df['trans_betwn_herds'].values, weights=w)
        true_trans_neighbor = np.average(df['trans_neighbor'].values, weights=w)
    ```
    - This loop iterates over the populations in the ABC-SMC history object `h`.
    - It retrieves the distribution of parameters and their weights for the specified model (`m=0`).
    - It calculates the weighted average of the parameters `trans`, `trans_betwn_herds`, and `trans_neighbor`.

4. **Defining Ground Truth and Estimated Values**:
    ```python
    gt_true = {
        "trans": true_trans, 
        "trans_betwn_herds": true_trans_betwn_herds, 
        "trans_neighbor": true_trans_neighbor
    }
    
    gt_est = {
        "trans": trans_est, 
        "trans_betwn_herds": trans_betwn_herds_est, 
        "trans_neighbor": trans_neighbor_est
    }
    ```
    - `gt_true`: Dictionary of true parameter values.
    - `gt_est`: Dictionary of estimated parameter values.

5. **Plotting the KDE Matrix**:
    ```python
    dfw = h.get_distribution(m=0, t=i)
    plot_kde_matrix(
        dfw, 
        dfw, 
        limits={
            "trans": (0, max_trans), 
            "trans_betwn_herds": (0, max_trans_betwn_herds),
            "trans_neighbor": (0, max_trans_neighbor)
        },
        refval=gt_est,
        refval_color='k',
        height=3.0,
        names={
            "trans": r"$\beta_{int}$", 
            "trans_betwn_herds": r"$\beta_{ext}$", 
            "trans_neighbor": r"$\beta_{neigh}$"
        },
    )
    ```
    - `dfw`: Retrieves the distribution of parameters for the specified time point.
    - `plot_kde_matrix`: Plots the KDE matrix with specified limits, reference values, colors, and LaTeX names for axes labels.

6. **Customizing the Plot**:
    ```python
    axes = plt.gcf().get_axes()
    for ax in axes:
        for line in ax.get_lines():
            line.set_linewidth(2.5)
        ax.tick_params(axis='both', which='major', labelsize=10, width=2)
        for spine in ax.spines.values():
            spine.set_linewidth(2)
        ax.spines['top'].set_visible(True)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.spines['right'].set_visible(True)
    ```
    - This block customizes the line widths, tick parameters, and spines for all subplots to enhance readability.

7. **Saving and Displaying the Plot**:
    ```python
    plt.savefig('kde_matrix_publication_thicker_lines_L2euclid_betas111.png', dpi=300, bbox_inches='tight')
    plt.show()
    ```
    - `plt.savefig`: Saves the plot as a high-resolution PNG file.
    - `plt.show`: Displays the plot.

This setup visualizes the KDE matrix for the specified parameters, customizing the plot for better readability and publication quality.


In [ ]:
from pyabc.visualization import plot_kde_matrix

# Parameter values to be estimated
trans_est = 1
trans_betwn_herds_est = 1
trans_neighbor_est = 1

# Iterate over populations
for i in range(h.max_t, h.max_t+1):
    # Estimated parameters
    df, w = h.get_distribution(m=0)
    true_trans = np.average(df['trans'].values, weights=w)
    true_trans_betwn_herds = np.average(df['trans_betwn_herds'].values, weights=w)
    true_trans_neighbor = np.average(df['trans_neighbor'].values, weights=w)
    
    # True values for ground truth (use original names as keys)
    gt_true = {
        "trans": true_trans, 
        "trans_betwn_herds": true_trans_betwn_herds, 
        "trans_neighbor": true_trans_neighbor
    }
    
    # Estimated values for reference (use original names as keys)
    gt_est = {
        "trans": trans_est, 
        "trans_betwn_herds": trans_betwn_herds_est, 
        "trans_neighbor": trans_neighbor_est
    }
    
    # KDE matrix plot with LaTeX names for axes labels
    dfw = h.get_distribution(m=0, t=i)  # First samples
    plot_kde_matrix(
        dfw[0], 
        dfw[1], 
        limits={
            "trans": (0, max_trans), 
            "trans_betwn_herds": (0, max_trans_betwn_herds),
            "trans_neighbor": (0, max_trans_neighbor)
        },
        refval=gt_est,  # Use original names for refval
        refval_color='k',  # Reference values in black
        height=3.0,  # Increase height for better readability in publication
        names={
            "trans": r"$\beta_{int}$", 
            "trans_betwn_herds": r"$\beta_{ext}$", 
            "trans_neighbor": r"$\beta_{neigh}$"
        },
    )
    
    # Customize the line widths and axes for all subplots
    axes = plt.gcf().get_axes()  # Get all axes in the figure
    for ax in axes:
        # Set thicker line width for KDE plots (both 1D and 2D)
        for line in ax.get_lines():
            line.set_linewidth(2.5)
        
        # Thicker ticks for both x and y axes
        ax.tick_params(axis='both', which='major', labelsize=10, width=2)
        
        # Enclose all subplots with thicker spines (border of the axes)
        for spine in ax.spines.values():
            spine.set_linewidth(2)

        # Ensure all spines are visible
        ax.spines['top'].set_visible(True)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.spines['right'].set_visible(True)

# Save the figure
plt.savefig('kde_matrix_publication_thicker_lines_L2euclid_betas111.png', dpi=300, bbox_inches='tight')  # High-resolution save

plt.show()